In [104]:
##############################################  Train Sentiment Analysis  ##############################################
from sklearn.model_selection import train_test_split # function for splitting data to train and test sets
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

# Splitting the dataset into train and test set
train, test = train_test_split(data,test_size = 0.1)
# Removing neutral sentiments
train = train[train.sentiment != "Neutral"]

tweets = []
stopwords_set = set(stopwords.words("english"))

for index, row in train.iterrows():
    words_filtered = [e.lower() for e in row.text.split() if len(e) >= 3]
    words_cleaned = [word for word in words_filtered
        if 'http' not in word
        and not word.startswith('@')
        and not word.startswith('#')
        and word != 'RT']
    words_without_stopwords = [word for word in words_cleaned if not word in stopwords_set]
    tweets.append((words_cleaned,row.sentiment))

test_pos = test[ test['sentiment'] == 'Positive']
test_pos = test_pos['text']
test_neg = test[ test['sentiment'] == 'Negative']
test_neg = test_neg['text']

# Extracting word features
def get_words_in_tweets(tweets):
    all = []
    for (words, sentiment) in tweets:
        all.extend(words)
    return all

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    features = wordlist.keys()
    return features
w_features = get_word_features(get_words_in_tweets(tweets))

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in w_features:
        features['containts(%s)' % word] = (word in document_words)
    return features

# Training the Naive Bayes classifier
training_set = nltk.classify.apply_features(extract_features,tweets)
classifier = nltk.NaiveBayesClassifier.train(training_set)
##############################################  Train Sentiment Analysis  ##############################################

In [119]:
##############################################  Sentiment Analysis Function  ##############################################
from textblob import TextBlob
import re #regular #remove all the useless words in analytics

def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())#Inputing the clean tweets

def analize_sentiment(tweet):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(clean_tweet(tweet))
    
    #This part is to transform scores to 1 or 0 to identify positive or negative
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1
##############################################  Sentiment Analysis Function  ##############################################

In [1]:
#!pip install selenium

In [5]:
import tweepy
import json
import math
import glob
import csv
import zipfile
import zlib
from tweepy import TweepError

In [80]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from time import sleep
import json
import datetime

In [81]:
def format_day(date):
    day = '0' + str(date.day) if len(str(date.day)) == 1 else str(date.day)
    month = '0' + str(date.month) if len(str(date.month)) == 1 else str(date.month)
    year = str(date.year)
    return '-'.join([year, month, day])

###################################   Note : How to change URL   ###################################
def form_url(user, since, until):
    p1 = 'https://twitter.com/search?f=tweets&vertical=default&q=%22'
    p2 =  user + '%22%20since%3A' + since + '%20until%3A' + until + '%20lang%3Aen&src=typd'
    return p1 + p2
###################################   Note : How to change URL   ###################################

def increment_day(date, i):
    return date + datetime.timedelta(days=i)

def is_retweet(entry):
    return 'retweeted_status' in entry.keys()

def get_source(entry):
    if '<' in entry["source"]:
        return entry["source"].split('>')[1].split('<')[0]
    else:
        return entry["source"]

In [6]:
######################################### Setting Twitter Account #########################################
consumer_key = 'RRqhLxEzw5RhnGjKoDRUA'
consumer_secret ='r9t9toH1yDrVwqAOlxxUtAfteVmGYJJHp7o8xAaBM4'
access_token = '280377182-U33H6LT1Fz0Ai5CQdzeanqGZyj19IBExuaXhmGZm'
access_token_secret = 'hWwIxPcpypyq548eyhewMrKrTlJvkJQRvCbY4KKo5BlfO'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
######################################### Setting Twitter Account #########################################

In [171]:
def Get_twitter_data(keywards,start,end):
    
    ################################# Set Keywords, since, until, output file names #################################
    # CHANGE THIS TO THE USER YOU WANT
    # edit these three variables
    #keywards = 'sphygmomanometer'
    user = keywards
    #start = datetime.datetime(2016, 12, 7)  # year, month, day
    #end = datetime.datetime(2016, 12, 8)  # year, month, day
    start_date=str(start).split(' ')[0]
    user = user.lower()

    output_file = '{}.json'.format(user.split("\"")[1]+start_date)
    output_file_short = '{}_short.json'.format(user.split("\"")[1]+start_date)
    compression = zipfile.ZIP_DEFLATED
    ################################# Set Keywords, since, until, output file names #################################
    
    ################################################ Getting Tweets IDs ################################################
    # only edit these if you're having problems
    delay = 1  # time to wait on each page load before reading the page
    driver = webdriver.Chrome("C:\Driver\chromedriver.exe")  # options are Chrome() Firefox() Safari()

    # don't mess with this stuff
    twitter_ids_filename = 'all_ids_{}.json'.format(user.split("\"")[1]+'_'+start_date)
    days = (end - start).days + 1
    id_selector = '.time a.tweet-timestamp'
    tweet_selector = 'li.js-stream-item'
    user = user.lower()
    ids = []

    #set id files with driver
    for day in range(days):
        d1 = format_day(increment_day(start, 0))
        d2 = format_day(increment_day(start, 1))
        url = form_url(user, d1, d2)
        print(url)
        print(d1)
        driver.get(url)
        sleep(delay)

        try:
            found_tweets = driver.find_elements_by_css_selector(tweet_selector)
            increment = 10

            while len(found_tweets) >= increment:
                print('scrolling down to load more tweets')
                driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                sleep(delay)
                found_tweets = driver.find_elements_by_css_selector(tweet_selector)
                increment += 10

            print('{} tweets found, {} total'.format(len(found_tweets), len(ids)))

            for tweet in found_tweets:
                try:
                    id = tweet.find_element_by_css_selector(id_selector).get_attribute('href').split('/')[-1]
                    ids.append(id)
                except StaleElementReferenceException as e:
                    print('lost element reference', tweet)

        except NoSuchElementException:
            print('no tweets on this day')

        start = increment_day(start, 1)

    ################ Show the condition when scraping is done ################
    try:
        with open(twitter_ids_filename) as f:
            all_ids = ids + json.load(f)
            data_to_write = list(set(all_ids))
            print('tweets found on this scrape: ', len(ids))
            print('total tweet count: ', len(data_to_write))
    except FileNotFoundError:
        with open(twitter_ids_filename, 'w') as f:
            all_ids = ids
            data_to_write = list(set(all_ids))
            print('tweets found on this scrape: ', len(ids))
            print('total tweet count: ', len(data_to_write))

    with open(twitter_ids_filename, 'w') as outfile:
        json.dump(data_to_write, outfile)

    print('all done here')
    driver.close()
    ################################################ Getting Tweets IDs ################################################
    
    ###################################### Create and Zip Files ######################################
    with open(twitter_ids_filename) as f:
        ids = json.load(f)

    print('total ids: {}'.format(len(ids)))

    all_data = []
    start = 0
    end = 100
    limit = len(ids)
    i = math.ceil(limit / 100)

    for go in range(i):
        print('currently getting {} - {}'.format(start, end))
        sleep(6)  # needed to prevent hitting API rate limit
        id_batch = ids[start:end]
        start += 100
        end += 100
        tweets = api.statuses_lookup(id_batch)
        for tweet in tweets:
            all_data.append(dict(tweet._json))

    print('metadata collection complete')
    print('creating master json file')
    with open(output_file, 'w') as outfile:
        json.dump(all_data, outfile)

    print('creating ziped master json file')
    zf = zipfile.ZipFile('{}.zip'.format(user.split("\"")[1]+start_date), mode='w')
    zf.write(output_file, compress_type=compression)
    zf.close()
    ###################################### Create and Zip Files ######################################
    
    ###################################### Get Results ######################################
    results=[]
    with open(output_file) as json_data:
        data = json.load(json_data)
        for entry in data:
            t = {     
                "text": entry["text"],
                "Likes": entry["favorite_count"],
                "Retweet_count": entry["retweet_count"],      
                "is_retweet": is_retweet(entry)
            }
            results.append(t)
        display(results)

    print('creating minimized json master file')
    with open(output_file_short, 'w') as outfile:
        json.dump(results, outfile)
    ###################################### Get Results ######################################
    
    ###################################### Get Results DataFrame ######################################
    scraping_twitter_data = pd.DataFrame(data=results)
    scraping_twitter_data['SA'] = np.array([analize_sentiment(tweet) for tweet in scraping_twitter_data['text']])  
    scraping_twitter_data['SA_Likes']=scraping_twitter_data['SA']*scraping_twitter_data['Likes']
    scraping_twitter_data['SA_RTs']=scraping_twitter_data['SA']*scraping_twitter_data['Retweet_count']
    scraping_twitter_data['SA_is_retweet']=scraping_twitter_data['SA']*scraping_twitter_data['is_retweet']
    #save the result
    scraping_twitter_data.to_csv('{}.csv'.format(keywards.split("\"")[1]+'_'+start_date),index=False)

    TweetsNum=len(scraping_twitter_data)
    Total_Likes=sum(scraping_twitter_data['Likes'])
    Total_RTs=sum(scraping_twitter_data['Retweet_count'])
    Sum_SA_Likes=sum(scraping_twitter_data['SA_Likes'])
    Sum_SA_RTs=sum(scraping_twitter_data['SA_RTs'])
    Sum_SA_is_retweet=sum(scraping_twitter_data['SA_is_retweet'])
    Num_is_RTs=len(scraping_twitter_data[scraping_twitter_data['is_retweet']==True])
    Positive=len(scraping_twitter_data[scraping_twitter_data['SA']==1])/TweetsNum
    Negative=len(scraping_twitter_data[scraping_twitter_data['SA']==-1])/TweetsNum
    ###################################### Get Results DataFrame ######################################
    
    return (TweetsNum,Total_Likes,Total_RTs,Sum_SA_Likes,Sum_SA_RTs,Sum_SA_is_retweet,Num_is_RTs,Positive,Negative)

In [179]:
import math

In [256]:
def slice_and_save_df(df,n=10):
    for i in range(0,math.ceil((len(df))/n)):
        if i==(math.ceil((len(df))/n)-1):
            slice_df=df[i*n:]
            res=fulfill_df(slice_df,i*n)
            res.to_csv('MovieRevenue_All_slice0_fromRowAvatar_slice{}.csv'.format(str(i)),index=True)
            #scraping_twitter_data.to_csv('{}.csv'.format(keywards.split("\"")[1]+'_'+start_date),index=False)
        else:
            slice_df=df[i*n:(i+1)*(n)]
            res=fulfill_df(slice_df,i*n)
            res.to_csv('MovieRevenue_All_slice0_fromRowAvatar_slice{}.csv'.format(str(i)),index=True)
        

In [255]:
def fulfill_df(df,start_from):
    
    df['TweetsNum']=0
    df['Total_Likes']=0
    df['Total_RTs']=0
    df['Sum_SA_Likes']=0
    df['Sum_SA_RTs']=0
    df['Sum_SA_is_retweet']=0
    df['Num_is_RTs']=0
    df['Positive']=0.0
    df['Negative']=0.0

    for i in range(0,len(df)):
        i=i+start_from
        res=Get_twitter_data(("\""+df['Movie_Name'][i]+"\""),
                             datetime.datetime.strptime(df['since_str'][i],"%Y-%m-%d"),
                             datetime.datetime.strptime(df['until_str'][i],"%Y-%m-%d"))

        df['TweetsNum'][i]=res[0]
        df['Total_Likes'][i]=res[1]
        df['Total_RTs'][i]=res[2]
        df['Sum_SA_Likes'][i]=res[3]
        df['Sum_SA_RTs'][i]=res[4]
        df['Sum_SA_is_retweet'][i]=res[5]
        df['Num_is_RTs'][i]=res[6]
        df['Positive'][i]=res[7]
        df['Negative'][i]=res[8]
    return df

# Start Catching Data

In [250]:
data0=pd.read_csv('MovieRevenue_All_v3.csv',encoding = "ISO-8859-1")

In [252]:
data1=data0[:]
data1=data1.reset_index(drop=True)

In [254]:
data1.head(1)

,index,Movie_Name,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year,since_str,since_date,...,Month,Day,Genre,Director,Actors,Rating,Revenue,Runtime,Voting,Metascore
0,177,Avatar,0.0,0.00,0.0,0.0,-7.0,2009,2009-10-23,2009-10-23 00:00:00,...,10,23,"Action,Adventure,Fantasy",James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",7.8,760.51,162.0,935408.0,83.0
1,178,Avatar,0.0,0.00,0.0,0.0,-6.0,2009,2009-10-30,2009-10-30 00:00:00,...,10,30,"Action,Adventure,Fantasy",James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",7.8,760.51,162.0,935408.0,83.0
2,179,Avatar,0.0,0.00,0.0,0.0,-5.0,2009,2009-11-06,2009-11-06 00:00:00,...,11,6,"Action,Adventure,Fantasy",James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",7.8,760.51,162.0,935408.0,83.0
3,180,Avatar,0.0,0.00,0.0,0.0,-4.0,2009,2009-11-13,2009-11-13 00:00:00,...,11,13,"Action,Adventure,Fantasy",James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",7.8,760.51,162.0,935408.0,83.0
4,181,Avatar,0.0,0.00,0.0,0.0,-3.0,2009,2009-11-20,2009-11-20 00:00:00,...,11,20,"Action,Adventure,Fantasy",James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",7.8,760.51,162.0,935408.0,83.0
5,182,Avatar,0.0,0.00,0.0,0.0,-2.0,2009,2009-11-27,2009-11-27 00:00:00,...,11,27,"Action,Adventure,Fantasy",James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",7.8,760.51,162.0,935408.0,83.0
6,183,Avatar,0.0,0.00,0.0,0.0,-1.0,2009,2009-12-04,2009-12-04 00:00:00,...,12,4,"Action,Adventure,Fantasy",James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",7.8,760.51,162.0,935408.0,83.0
7,184,Avatar,0.0,0.00,0.0,0.0,0.0,2009,2009-12-11,2009-12-11 00:00:00,...,12,11,"Action,Adventure,Fantasy",James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",7.8,760.51,162.0,935408.0,83.0
8,185,Avatar,137094001.0,0.00,39714.0,137094051.0,1.0,2009,2009-12-18,2009-12-18 00:00:00,...,12,18,"Action,Adventure,Fantasy",James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",7.8,760.51,162.0,935408.0,83.0
9,186,Avatar,146530209.0,0.07,42399.0,283624210.0,2.0,2009,2009-12-25,2009-12-25 00:00:00,...,12,25,"Action,Adventure,Fantasy",James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",7.8,760.51,162.0,935408.0,83.0


In [ ]:
slice_and_save_df(data1)